In [ ]:
%ls

In [ ]:
import os
import csv 
import time
import copy
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from scipy import signal
import librosa
import librosa.display

#from pytorch_model_summary import summary
from tqdm import trange
from torch.utils.data import TensorDataset, DataLoader
from torchsummary import summary

plt.rcParams["figure.figsize"] = (5, 5)

In [ ]:
from Dreem_functions.visualisation import SIGNALS_NAME, extract_events_from_binary_mask, \
                                          visualize_signal_and_event, visualise_index
from Dreem_functions.metric_dreem  import jaccard_overlap, extract_events_from_binary_mask, \
                                          compute_f1_score, compute_tp_fp_fn_for_each_entry,\
                                          format_predictions_for_scoring, dreem_sleep_apnea_custom_metric
from Our_functions.create_tensors  import normalize_dim, normalize_ppson, compute_spectrograms
from Our_functions.create_datasets import create_dataset, create_3_part_dataset, create_4_part_dataset, \
                                          create_loader
from Our_functions.raw_models      import RawCNN, GruCNN
from Our_functions.new_raw_models  import Unet
from Our_functions.fft_models      import fftCNN
from Our_functions.dual_models     import DualCNN
from Our_functions.loss_functions  import DiceBCELoss, TverskyLoss, FocalLoss

In [ ]:
# Visualize signals

plt.rcParams["figure.figsize"] = (18, 9)

PATH_TO_TRAINING_DATA = "Data sets/additional_files_dreem/X_train.h5"
PATH_TO_TRAINING_TARGET = "Data sets/Training/y_train_tX9Br0C.csv"
h5_file = h5py.File(PATH_TO_TRAINING_DATA)
mask = np.array(pd.read_csv(PATH_TO_TRAINING_TARGET))
visualise_index(0, h5_file, mask)
visualise_index(18, h5_file, mask)
visualise_index(52, h5_file, mask)

In [ ]:
# Create training data
print("Train set...")

PATH_TO_TRAINING_DATA = "Data sets/additional_files_dreem/X_train.h5"
train_file = h5py.File(PATH_TO_TRAINING_DATA)

train_set = train_file['data'][:, 2:]

train_set_raw = train_set.reshape((4400, 8, 9000))
#train_set_fft = compute_spectrograms(train_set_raw, status='mel', verbose=True)
train_set_raw = normalize_ppson(train_set_raw)


# Create testing data
print("Test  set...")

PATH_TO_TESTING_DATA = "Data sets/additional_files_dreem/X_test.h5"
test_file = h5py.File(PATH_TO_TESTING_DATA)

test_set = test_file['data'][:, 2:]

test_set_raw = test_set.reshape((4400, 8, 9000))
#test_set_fft = compute_spectrograms(test_set_raw, status='mel', verbose=False)
test_set_raw = normalize_ppson(test_set_raw)


# Create training label
PATH_TO_TRAINING_TARGET = "Data sets/Training/y_train_tX9Br0C.csv"
masks = np.array(pd.read_csv(PATH_TO_TRAINING_TARGET))
masks = np.array(masks[:, 1:])

print("1D Input  shapes: {}".format(train_set_raw.shape))
#print("2D Input  shapes: {}".format(train_set_fft.shape))
print("Output    shapes: {}".format(masks.shape))

In [ ]:
train_sig0 = train_set_raw[:, 0, :].reshape((4400, 1, 9000))
train_sig1 = train_set_raw[:, 1, :].reshape((4400, 1, 9000))
train_sig2 = train_set_raw[:, 2, :].reshape((4400, 1, 9000))
train_sig3 = train_set_raw[:, 3, :].reshape((4400, 1, 9000))
train_sig4 = train_set_raw[:, 4, :].reshape((4400, 1, 9000))
train_sig5 = train_set_raw[:, 5, :].reshape((4400, 1, 9000))
train_sig6 = train_set_raw[:, 6, :].reshape((4400, 1, 9000))
train_sig7 = train_set_raw[:, 7, :].reshape((4400, 1, 9000))

test_sig0 = test_set_raw[:, 0, :].reshape((4400, 1, 9000))
test_sig1 = test_set_raw[:, 1, :].reshape((4400, 1, 9000))
test_sig2 = test_set_raw[:, 2, :].reshape((4400, 1, 9000))
test_sig3 = test_set_raw[:, 3, :].reshape((4400, 1, 9000))
test_sig4 = test_set_raw[:, 4, :].reshape((4400, 1, 9000))
test_sig5 = test_set_raw[:, 5, :].reshape((4400, 1, 9000))
test_sig6 = test_set_raw[:, 6, :].reshape((4400, 1, 9000))
test_sig7 = test_set_raw[:, 7, :].reshape((4400, 1, 9000))

print(train_sig0.shape)

In [ ]:
new_train_set = np.concatenate((train_sig0, train_sig1, train_sig2, train_sig3, train_sig5), axis=1)
new_test_set  = np.concatenate((test_sig0, test_sig1, test_sig2, test_sig3, test_sig5), axis=1)

print(new_train_set.shape)

In [ ]:
# Create dataset. Splitting into 5 for ensemble method.
train_dataset_dual = create_dataset(train_set_raw, masks)

splits_dual = torch.utils.data.random_split(train_dataset_dual, [880, 880, 880, 880, 880], generator=torch.Generator().manual_seed(11))
print(splits_dual)

In [ ]:
rawmodel = RawCNN().cuda()

summary(rawmodel, (8, 9000))

In [ ]:
grumodel = GruCNN().cuda()

summary(grumodel, (8, 9000))

In [ ]:
unetmodel = Unet().cuda()

summary(unetmodel, (8, 9000))

In [ ]:
fftmodel = fftCNN().cuda()

summary(fftmodel, (8, 8, 900))

In [ ]:
def train_model(model, train_loader, val_loader, model_index):
    model_file = 'Results/new_experiment' + '/' + 'grucnn_' + str(model_index) + '.pth'
    
    criterion = TverskyLoss() # alternative: look at loss_functions.py or torch.nn.BCELoss()
    #criterion = nn.BCELoss()
    
    lr = 0.0001
        
    since = time.time()
    num_epochs = 300
    
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

    best_acc = 0.

    for epoch in range(num_epochs):
        epoch_acc = 0.0

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                
                full_loss = 0.
                full_acc  = 0.
                
                for batch_idx, (raw, target) in enumerate(train_loader):
                    if use_cuda:
                        raw, target = raw.cuda(), target.cuda()

                    optimizer.zero_grad()
                    
                    # Input varies depending on network used
                    output = model(raw)
                    
                    loss = criterion(output, target)
                    loss.backward()
                    optimizer.step()
                    
                    full_loss += loss

                    cpu_output       = output.cpu().detach().numpy()
                    cpu_output       = np.rint(cpu_output)
                    cpu_output       = cpu_output.astype(int)
                    
                    try:
                        cpu_output   = format_predictions_for_scoring(cpu_output)
                    except:
                        cpu_output   = 0.
                    
                    cpu_target       = target.cpu().detach().numpy()
                    cpu_target       = np.rint(cpu_target)
                    cpu_target       = cpu_target.astype(int)
                    
                    try:
                        cpu_target   = format_predictions_for_scoring(cpu_target)
                    except:
                        cpu_target   = 0.
                    
                    full_acc  += compute_f1_score(cpu_output, cpu_target)
                
                full_loss = full_loss / (batch_idx + 1)
                full_acc  = full_acc  / (batch_idx + 1)

            else:
                model.eval()
                
                # Compute validation loss
                validation_loss = 0
                validation_acc  = 0
                
                for batch_idx, (raw, target) in enumerate(val_loader):
                    if use_cuda:
                        raw, target = raw.cuda(), target.cuda()

                    # Input varies depending on network used
                    output = model(raw)
                    
                    validation_loss += criterion(output, target).item()
                    
                    cpu_output       = output.cpu().detach().numpy()
                    cpu_output       = np.rint(cpu_output)
                    cpu_output       = cpu_output.astype(int)
                    
                    try:
                        cpu_output   = format_predictions_for_scoring(cpu_output)
                    except:
                        cpu_output   = 0.
                    
                    cpu_target       = target.cpu().detach().numpy()
                    cpu_target       = np.rint(cpu_target)
                    cpu_target       = cpu_target.astype(int)
                    
                    try:
                        cpu_target   = format_predictions_for_scoring(cpu_target)
                    except:
                        cpu_target   = 0.
                    
                    validation_acc  += compute_f1_score(cpu_output, cpu_target)
                    
                validation_loss = validation_loss / (batch_idx + 1)
                validation_acc  = validation_acc  / (batch_idx + 1)

                print('Epoch {} --> train set: loss is {:.3f}. F1 score is {:.3f}, validation set: loss is {:.3f}. F1 score is: {:.3f}'.format(epoch, full_loss, full_acc, validation_loss, validation_acc))
                
                epoch_acc = validation_acc
            
            # Save model if F1 score is better
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), model_file)
                print('--> Saved model to ' + model_file)
            
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60))

    return model

In [ ]:
# Create experiment folder
if not os.path.isdir('Results/new_experiment'):
    os.makedirs('Results/new_experiment')

batch_size = 100

use_cuda = torch.cuda.is_available()

for x in range(5):
    l = []

    for i in range(5):
        if (i == x):
            continue
        else:
            l.append(splits_dual[i])

    models = GruCNN()

    if (use_cuda == True):
      models.cuda()
    
    set_to_use = torch.utils.data.ConcatDataset(l)

    train_loader = torch.utils.data.DataLoader(set_to_use, batch_size=batch_size, shuffle=False, num_workers=1)
    val_loader = torch.utils.data.DataLoader(splits_dual[x], batch_size=batch_size, shuffle=False)    
    models = train_model(models, train_loader, val_loader, x)

In [ ]:
use_cuda = torch.cuda.is_available()

model1 = GruCNN()
state_dict1 = torch.load('Results/new_experiment/grucnn_0.pth')
model1.load_state_dict(state_dict1)
model1.eval()

model2 = GruCNN()
state_dict2 = torch.load('Results/new_experiment/grucnn_1.pth')
model2.load_state_dict(state_dict2)
model2.eval()

model3 = GruCNN()
state_dict3 = torch.load('Results/new_experiment/grucnn_2.pth')
model3.load_state_dict(state_dict3)
model3.eval()

model4 = GruCNN()
state_dict4 = torch.load('Results/new_experiment/grucnn_3.pth')
model4.load_state_dict(state_dict4)
model4.eval()

model5 = GruCNN()
state_dict5 = torch.load('Results/new_experiment/grucnn_4.pth')
model5.load_state_dict(state_dict5)
model5.eval()

outfile = 'Results/new_experiment/final_gru_results.csv'

if use_cuda:
    print('Using GPU')
    model1.cuda()
    model2.cuda()
    model3.cuda()
    model4.cuda()
    model5.cuda()
else:
    print('Using CPU')

output_file = open(outfile, "w")
output_file.write("ID,y_0,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12,y_13,y_14,y_15,y_16,y_17,y_18,y_19,"
                     "y_20,y_21,y_22,y_23,y_24,y_25,y_26,y_27,y_28,y_29,y_30,y_31,y_32,y_33,y_34,y_35,y_36,y_37,"
                     "y_38,y_39,y_40,y_41,y_42,y_43,y_44,y_45,y_46,y_47,y_48,y_49,y_50,y_51,y_52,y_53,y_54,y_55,"
                     "y_56,y_57,y_58,y_59,y_60,y_61,y_62,y_63,y_64,y_65,y_66,y_67,y_68,y_69,y_70,y_71,y_72,y_73,"
                     "y_74,y_75,y_76,y_77,y_78,y_79,y_80,y_81,y_82,y_83,y_84,y_85,y_86,y_87,y_88,y_89\n")
thresh = 0.5

for f in trange(test_set_raw.shape[0]):
    raw = test_set_raw[f, :, :]
    raw = raw.reshape((1, raw.shape[0], raw.shape[1]))
    raw = torch.tensor(raw, dtype=torch.float)
    
    #fft = test_set_fft[f, :, :, :]
    #fft = raw.reshape((1, fft.shape[0], fft.shape[1], fft.shape[2]))
    #fft = torch.tensor(fft, dtype=torch.float)

    if use_cuda:
        raw = raw.cuda()
    
    output1 = model1(raw)[0].cpu().detach().numpy()
    output1 = np.rint(output1)
    output1 = output1.astype(int)
    
    output2 = model2(raw)[0].cpu().detach().numpy()
    output2 = np.rint(output2)
    output2 = output2.astype(int)
    
    output3 = model3(raw)[0].cpu().detach().numpy()
    output3 = np.rint(output3)
    output3 = output3.astype(int)
    
    output4 = model4(raw)[0].cpu().detach().numpy()
    output4 = np.rint(output4)
    output4 = output4.astype(int)
    
    output5 = model5(raw)[0].cpu().detach().numpy()
    output5 = np.rint(output5)
    output5 = output5.astype(int)

    sum = (output1 + output2 + output3 + output4 + output5) / 5
    sum = np.rint(sum).astype(int)

    my_list = [4400+f] + list(sum)
    my_string = ','.join(map(str, my_list)) 

    output_file.write("%s\n" % (my_string))

output_file.close()

print("\nSuccesfully wrote " + outfile + ', you can upload this file to the challenge competition website')

In [ ]:
# Comparison with baseline
if __name__ == '__main__':
    CSV_FILE_Y_TRUE = 'Data sets/Results/y_benchmark.csv'
    CSV_FILE_Y_PRED = 'Results/new_experiment/final_gru_results.csv'
    df_y_true = pd.read_csv(CSV_FILE_Y_TRUE, index_col=0, sep=',')
    df_y_pred = pd.read_csv(CSV_FILE_Y_PRED, index_col=0, sep=',')
    print(dreem_sleep_apnea_custom_metric(df_y_true, df_y_pred))